# Tutorial: Criando um chat bot simples com a API Gemini

Este notebook guia voce passo a passo na construcao de um chat bot que utiliza a plataforma Gemini da Google para responder perguntas em portugues. A proposta e que voce entenda cada etapa - da instalacao a criacao de um laco interativo - e adapte o codigo as suas necessidades.


## Visao geral

Ao final, voce tera:
- uma configuracao segura da sua chave de API;
- um cliente da API pronto para enviar perguntas isoladas;
- um chat com memoria contextual;
- um laco interativo completo, incluindo historico das mensagens.

Se preferir executar em outro ambiente (VS Code, Colab, etc.), os passos permanecem os mesmos.


## Pre-requisitos

- Python 3.10 ou superior.
- Uma conta no Google AI Studio com acesso ao modelo Gemini.
- Uma chave de API ativa e com permissoes para o modelo `gemini-2.5-flash`.
- Biblioteca `google-genai` instalada (mostramos a instalacao no proximo passo).


## 1. Instalar dependencias

Se voce estiver em um notebook (Jupyter, Colab, etc.), execute a celula abaixo para instalar ou atualizar a biblioteca `google-genai`. Em ambientes gerenciados, lembre-se de verificar as politicas de instalacao antes de rodar comandos.


In [ ]:
%pip install --quiet --upgrade google-genai

## 2. Configurar a sua chave de API

Armazene a chave em uma variavel de ambiente para evitar expor o valor diretamente no codigo. No notebook, voce pode defini-la manualmente (apenas para testes) ou usar cofres/segredos da plataforma.

> Dica: Se estiver no Google Colab, substitua a leitura da variavel por `from google.colab import userdata` e recupere o segredo com `userdata.get("NOME_DO_SEGREDO")`.


In [ ]:
import os

API_KEY = os.getenv('GOOGLE_API_KEY', 'COLE_SUA_CHAVE_AQUI')
if API_KEY == 'COLE_SUA_CHAVE_AQUI':
    raise ValueError(
        "Defina a variavel de ambiente GOOGLE_API_KEY ou substitua 'COLE_SUA_CHAVE_AQUI' pelo valor real da sua chave."
    )

os.environ['GOOGLE_API_KEY'] = API_KEY
print('Chave configurada com sucesso.')


## 3. Conectar ao modelo Gemini

Com a chave disponivel, criamos um cliente para conversar com o modelo. Execute esta etapa apenas uma vez por sessao.


In [ ]:
from google import genai

client = genai.Client()
print('Cliente inicializado com sucesso.')


## 4. Primeiro teste: pergunta isolada

Use `generate_content` para enviar perguntas pontuais. E uma forma rapida de validar credenciais e latencia antes de partir para o modo de chat.


In [ ]:
pergunta = input('Digite uma pergunta rapida para o modelo: ')
resposta = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=pergunta,
)
print('Resposta:\n', resposta.text)


## 5. Iniciar um chat com memoria contextual

Crie uma sessao de chat persistente para que o modelo considere o historico de mensagens antes de responder.


In [ ]:
chat = client.chats.create(model='gemini-2.5-flash')
print('Chat iniciado. Faca uma pergunta inicial para abrir a conversa.')
primeira_pergunta = input('Voce: ')
primeira_resposta = chat.send_message(primeira_pergunta)
print('Gemini:\n', primeira_resposta.text)


## 6. Construir um laco interativo

A funcao abaixo mantem o dialogo aberto ate voce digitar `fim`. Ela tambem ignora entradas vazias para evitar chamadas desnecessarias a API.


In [ ]:
def conversar(chat_session):
    print("\nDigite 'fim' para encerrar a conversa.\n")
    while True:
        mensagem = input('Voce: ').strip()
        if not mensagem:
            print('Mensagem vazia ignorada.')
            continue
        if mensagem.lower() == 'fim':
            print('Encerrando a conversa.')
            break

        resposta = chat_session.send_message(mensagem)
        print('Gemini:\n', resposta.text, '\n')

conversar(chat)


## 7. Consultar o historico de mensagens

Depois da conversa, recupere o historico completo para salvar logs, gerar analises ou alimentar outras ferramentas.


In [ ]:
historico = chat.get_history()

for indice, turno in enumerate(historico, start=1):
    papel = turno.role.capitalize()
    partes = getattr(turno, 'parts', [])
    conteudo = '
'.join(
        getattr(parte, 'text', '')
        for parte in partes
        if getattr(parte, 'text', '')
    ) or '[mensagem vazia]'
    print(f"{indice}. {papel}: {conteudo}
")


## Proximos passos

- Troque o modelo para variantes mais poderosas (`gemini-2.0-pro`, etc.) conforme a necessidade.
- Trate erros e limites de taxa antes de levar o chat para producao.
- Conecte este backend a uma interface (web, CLI, WhatsApp) reaproveitando as funcoes apresentadas aqui.

Bom estudo!
